In [208]:
from scipy.optimize import rosen,rosen_der, minimize_scalar, minimize
import numpy as np

# Powell's Method (without derivatives)

In [32]:
def powell(x0, func,params, rtol=1e-6, maxiter=None):
    """
    Minimize a function using the Powell's method.

    Parameters:
    ----------
    func : callable
        Function to minimize. Should take a 1d numpy array as input and return a scalar.
    x0 : 1d numpy array
        Initial guess for the minimum.
    tol : float
        Tolerance for termination.
    maxiter : int or None
        Maximum number of iterations. None means no limit.

    Returns:
    -------
    x : 1d numpy array
        The minimum value found.
    fval : float
        The function value at the minimum.
    """

    # Define the number of variables and directions
    n = len(x0)
    direc = np.eye(n)

    # Define the initial position and function value
    x = x0.copy()
    fval = func(x,params)

    # Define the maximum number of iterations
    if maxiter is None:
        maxiter = n * 1000

    # Iterate until convergence or maximum iterations
    for i in range(maxiter):
        # Store the previous position and function value
        x_old = x.copy()
        fval_old = fval

        # Iterate over all directions
        for j in range(n):
            # Define the line search function along the current direction
            def line_search(alpha):
                return func(x + alpha * direc[j],params)

            # Find the minimum of the line search function
            res = minimize_scalar(line_search)

            # Update the current position and function value
            x = x + res.x * direc[j]
            fval = func(x,params)

        # Define the new set of directions based on the previous and current positions
        direc = np.vstack((direc[1:], (x - x_old) / np.linalg.norm(x - x_old)))

        # Check for convergence
        if 2*np.abs(fval - fval_old) < rtol*(np.abs(fval)+np.abs(fval_old))+1e-25:
            break

    return x, fval

# FIRE method

In [173]:
def fire2(x0,f,df,params,atol=1e-8,rtol=1e-6,dt = 0.002):
    alpha0 = 0.25
    Ndelay = 5
    finc = 1.1
    fdec = 0.5
    fa = 0.99
    Nnegmax = 2000
    
    n = len(x0)
    # Define the maximum number of iterations
    maxiter = n * 1000
    
    dtmax = 10*dt
    dtmin = 0.02*dt
    alpha = alpha0
    Npos = 0
    Nneg = 0

    x = x0.copy()
    V = np.zeros_like(x)
    fval = f(x,params)
    F = -df(x,params)

    for i in range(maxiter):

        P = (F*V).sum() # dissipated power
        
        if (P>0):
            Npos = Npos + 1
            Nneg = 0
            if Npos>Ndelay:
                dt = min(dt*finc,dtmax)
                alpha = alpha*fa
        else:
            Npos = 0
            Nneg = Nneg + 1
            if Nneg > Nnegmax: break
            if i> Ndelay:
                dt = max(dt*fdec,dtmin)
                alpha = alpha0
            x = x - 0.5*dt*V + 0.5*dt**2*F
            V = np.zeros_like(x)
            
        V = V + 0.5*dt*F
        V = (1-alpha)*V + alpha*F*np.linalg.norm(V)/np.linalg.norm(F)
        x = x + dt*V + 0.5*dt**2*F
        fval_old = fval
        fval = f(x,params)
        F = -df(x,params)
        V = V + 0.5*dt*F
        
        sk=atol+rtol*np.abs(x)
        error = np.linalg.norm(F/sk)
        if  error < 1.0: break
#         print(i,error)
 
    return x,fval

# Anderson Mixing

In [261]:
def anderson(x0,f,df,params,atol=1e-8,rtol=1e-6):
    n = len(x0)
    # Define the maximum number of iterations
    maxiter = n * 1000
    m = 3 # parameter m 
    x = x0.copy()
    F = np.empty_like(x)
    xpast = np.zeros((m,n))
    Fpast = np.zeros((m,n))
    alpha = np.ones(m)/m
    beta = 0.5
    
    for i in range(maxiter):
        
        F[:] = -df(x,params)
        
        if i < m: 
            Fbar = 0.01*F 
            xbar = x
        else: 
            Fbar = F + alpha@(Fpast-F)
            xbar = x + alpha@(xpast-x)
        
            def objective_function(alphasqrt):
                alpha = alphasqrt**2
                alpha = alpha/alpha.sum()
                return np.linalg.norm(F + alpha@(Fpast-F))
            
            # Find the minimum of the line search function
            res = minimize(objective_function, np.sqrt(alpha), method='Nelder-Mead', tol=1e-6)
            alpha[:] = res.x**2
        
        print(alpha,alpha.sum())
        
        x = xbar + beta*Fbar
        Fpast = np.vstack((Fpast[1:],F))
        xpast = np.vstack((xpast[1:],x))
        
        sk=atol+rtol*np.abs(x)
        error = np.linalg.norm(F/sk)
        if  error < 1.0: break
            
    fval = f(x,params)
            
    return x,fval

## Optimizing the Rosenbrock function 
$xmim=(1.0,1.0)$

$fmim=0.0$

In [237]:
# Define the Rosenbrock function
def rosenbrock(x,params=None):
    return rosen(x)

# Define the Rosenbrock function
def gradient_rosenbrock(x,params=None):
    return rosen_der(x)

# Define the initial guess
x0 = np.array([0.5, 0.5])

In [218]:
# Use the Powell's method to minimize the Rosenbrock function
x_min, f_min = powell(x0,rosenbrock,params=None,rtol=1e-6)

print("Minimum value found: ", x_min)
print("Function value at the minimum: ", f_min)

Minimum value found:  [0.71001923 0.50207097]
Function value at the minimum:  0.0845117006093497


In [219]:
rosenbrock(np.array([1.0,1.0]),None)

0.0

In [220]:
# Use the FIRE method to minimize the Rosenbrock function
x_min, f_min = fire2(x0,rosenbrock, gradient_rosenbrock,params='None',atol=1e-8,rtol=1e-6)

print("Minimum value found: ", x_min)
print("Function value at the minimum: ", f_min)

Minimum value found:  [0.99999981 0.99999962]
Function value at the minimum:  3.633732583072612e-14


In [221]:
%timeit -n 10 fire2(x0,rosenbrock, gradient_rosenbrock,params='None',atol=1e-12,rtol=1e-10)

46.6 ms ± 513 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [222]:
%timeit -n 10 powell(x0,rosenbrock,params=None,rtol=1e-6)

3.3 ms ± 158 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [262]:
# Use the Anderson method to minimize the Rosenbrock function
x_min, f_min = anderson(x0,rosenbrock, gradient_rosenbrock,params='None',atol=1e-8,rtol=1e-6)

print("Minimum value found: ", x_min)
print("Function value at the minimum: ", f_min)

[0.33333333 0.33333333 0.33333333] 1.0
[0.33333333 0.33333333 0.33333333] 1.0
[0.33333333 0.33333333 0.33333333] 1.0
[7.15794798e-01 4.40723089e-01 5.48527532e-14] 1.1565178875120166
[5.23843859e-01 5.33279830e-01 6.07491932e-14] 1.0571236885358297
[3.82029863e-01 6.45384670e-01 6.60269701e-14] 1.027414533282305
[4.16188706e-01 6.45384670e-01 6.60269701e-14] 1.0615733753679766
[4.37258259e-01 6.45384670e-01 6.60269701e-14] 1.08264292858475
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.936

/tmp/ipykernel_260622/1589260705.py:21: RuntimeWarning: invalid value encountered in add
  Fbar = F + alpha@(Fpast-F)
/tmp/ipykernel_260622/1589260705.py:27: RuntimeWarning: invalid value encountered in add
  return np.linalg.norm(F + alpha@(Fpast-F))


[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.5618589

[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.5618589

[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.5618589

[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.5618589

[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.5618589

[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.5618589

[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205
[4.44576262e-01 6.56185899e-01 6.93695855e-14] 1.100762160931205


KeyboardInterrupt: 

In [187]:
a = np.array([[1, 2, 3],[4,5,6]])
b = np.array([7, 8, 9])

In [200]:
alpha = np.array([1,1])

In [195]:
np.vstack((a,b))

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [202]:
alpha@(a+b)

array([19, 23, 27])

In [203]:
(a+b)

array([[ 8, 10, 12],
       [11, 13, 15]])